In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import xgboost as xgb
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_columns = 999

/home/amin/Software/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
print("Train shape : ", train.shape)
print("Test shape : ", test.shape)

('Train shape : ', (4209, 378))
('Test shape : ', (4209, 377))


In [4]:
# drop columns with unique value in train
for k in train.keys():
    if len(train[k].unique()) == 1:
        train.pop(k)
        test.pop(k)
        print "unique train" + k
        
# drop columns with unique value in test
for k in test.keys():
    if len(test[k].unique()) == 1:
        train.pop(k)
        test.pop(k)
        print "unique test" + k
        
print train.shape, test.shape

unique trainX11
unique trainX93
unique trainX107
unique trainX233
unique trainX235
unique trainX268
unique trainX289
unique trainX290
unique trainX293
unique trainX297
unique trainX330
unique trainX347
unique testX257
unique testX258
unique testX295
unique testX296
unique testX369
(4209, 361) (4209, 360)


In [5]:
# drop duplicate columns
col = train.T.drop_duplicates().T.columns.values.tolist()
train = train[col]
col.remove("y")
test = test[col]
print train.shape, test.shape

(4209, 317) (4209, 316)


In [6]:
dtype = train.dtypes.reset_index()
dtype.columns = ["Count", "Column Type"]
dtype.groupby("Column Type").aggregate('count').reset_index()

,Column Type,Count
0,int64,308
1,float64,1
2,object,8


In [7]:
# duplicate rows exclude y and ID
col.remove("ID")
sum(train.duplicated(col, keep=False)) # duplicate rows

515

In [121]:
col_group = train.groupby(col)['y'].agg({np.mean})
col_group['count'] = train.groupby(col).count()['y']

for index, row in col_group.iterrows():
    
    if row['count'] > 1 :
#        print index
        dic = {}
        ll = [[i] for i in row.name]
        for ii, cc in enumerate(col):
            dic[cc] = ll[ii]
    
        idx = train[col].isin(dic).mean(axis=1) == 1
        train.loc[idx, "y"] = row['mean']
    

In [122]:
def find_identical_idx(a, b, cols):
    idx = a[cols[0]].isin(b[cols[0]].values)
    if len(cols) > 1:
        for i in range(1,len(cols)):
            idx &= a[cols[i]].isin(b[cols[i]].values)
    return idx

In [123]:
tt = train[train.duplicated(col)]
ix = find_identical_idx(train, tt.iloc[:1], col)
train[ix]

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X36,X38,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X73,X74,X75,X77,X78,X79,X80,X81,X82,X83,X85,X86,X87,X88,X89,X90,X91,X92,X95,X96,X97,X98,X99,X100,X101,X103,X104,X105,X106,X108,X109,X110,X111,X112,X114,X115,X116,X117,X118,X120,X123,X124,X125,X126,X127,X128,X129,X130,X131,X132,X133,X135,X136,X137,X138,X139,X140,X141,X142,X143,X144,X145,X148,X150,X151,X152,X153,X154,X155,X156,X157,X158,X159,X160,X161,X162,X163,X164,X165,X166,X167,X168,X169,X170,X171,X173,X174,X175,X176,X177,X178,X179,X180,X181,X182,X183,X184,X185,X186,X187,X189,X190,X191,X192,X194,X195,X196,X197,X198,X200,X201,X202,X203,X204,X205,X206,X207,X208,X209,X210,X211,X212,X215,X217,X218,X219,X220,X221,X223,X224,X225,X228,X229,X230,X231,X234,X236,X237,X238,X240,X241,X246,X249,X250,X251,X252,X255,X256,X259,X260,X261,X263,X264,X265,X267,X269,X270,X271,X272,X273,X274,X275,X276,X277,X278,X280,X281,X282,X283,X284,X285,X286,X287,X288,X291,X292,X294,X298,X300,X301,X304,X305,X306,X307,X308,X309,X310,X311,X312,X313,X314,X315,X316,X317,X318,X319,X321,X322,X323,X325,X327,X328,X329,X331,X332,X333,X334,X335,X336,X337,X338,X339,X340,X341,X342,X343,X344,X345,X346,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X361,X362,X363,X366,X367,X368,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X383,X384
13,36,96.788,al,r,e,f,d,j,h,o,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
16,39,96.788,al,r,e,f,d,j,h,o,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
51,125,96.788,al,r,e,f,d,j,h,o,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
79,169,96.788,al,r,e,f,d,j,h,o,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0

In [130]:
new_train = train.drop_duplicates(subset=col, keep=False)

In [131]:
print("Train shape : ", new_train.shape)
print("Test shape : ", test.shape)

('Train shape : ', (3694, 317))
('Test shape : ', (4209, 316))


In [132]:
new_train.to_csv('clean_train.csv', index=False)

In [133]:
test.to_csv('clean_test.csv', index=False)